- 表达式(expression): 由标识符、字面量和操作符组成。其完成运算、属性访问以及函数运算等。表达式像数学公式总是返回一个结果
- 语句(statement): 由一到多行代码组成，其着重于逻辑过程，完成变量赋值、类型定义以及控制执行流方向等
- 表达式完成计算，语句执行逻辑。表达式是语句的一种，语句不一定是表达式
- eval执行表达式，exec执行语句

In [5]:
import sys
print(sys.flags.optimize) # 解释器参数
print(sys.argv) # 程序启动参数

0
['/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py', '-f', '/Users/hejl/Library/Jupyter/runtime/kernel-0f0cd9f4-73d1-47e8-8aa0-d51b94bed111.json']


- 终止进程的正确做法是调用sys.exit函数，它确保推出前会执行相关清理操作
- 常见的清理操作包括finally和atexit。前者是结构化异常子句，无论异常是否发生都会执行。后者用于注册进程退出前才执行的清理函数
- 终止进程应返回退出状态码（exit status），便于命令行工具据此做判断。返回0表示正常结束。

In [2]:
import atexit
# 这里交互式环境和命令行执行结果不同，原书在命令行执行的
atexit.register(print, 'haha')
try:
    exit
finally:
    print('finalss')

finalss


In [9]:
exit

haha
haha


- 帮助信息不同于注释，注释会被编译器忽略，帮助属于基本元数据，可在运行期查询和输出
- 帮助信息保存在\_\_doc__属性中，可通过help()方法查看
- 解释器以"OO"方式运行时，帮助信息会被移除

对于三个以内的变量交换，编译器会优化为ROT指令，直接交换栈帧内数据，而不是构建元组。

In [23]:
import dis
dis.dis(compile("a,b,c = c,b,a", "", "exec"))

  1           0 LOAD_NAME                0 (c)
              2 LOAD_NAME                1 (b)
              4 LOAD_NAME                2 (a)
              6 ROT_THREE
              8 ROT_TWO
             10 STORE_NAME               2 (a)
             12 STORE_NAME               1 (b)
             14 STORE_NAME               0 (c)
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [4]:
dis.dis(compile("a,b,c,d = d,c,b,a", "", "exec"))

  1           0 LOAD_NAME                0 (d)
              2 LOAD_NAME                1 (c)
              4 LOAD_NAME                2 (b)
              6 LOAD_NAME                3 (a)
              8 BUILD_TUPLE              4
             10 UNPACK_SEQUENCE          4
             12 STORE_NAME               3 (a)
             14 STORE_NAME               2 (b)
             16 STORE_NAME               1 (c)
             18 STORE_NAME               0 (d)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


星号收集不能单独出现，要么与其他名字一起，要么放入列表或元组内。

In [9]:
a, *b, c = 1,2
print(a,b,c)

1 [] 2


In [7]:
a, *b, c, d = 1,2

ValueError: not enough values to unpack (expected at least 3, got 2)

In [8]:
a, *b, *c, d = range(10)

SyntaxError: two starred expressions in assignment (<ipython-input-8-d74a602f44df>, line 1)

In [10]:
*a = 1,2

SyntaxError: starred assignment target must be in a list or tuple (<ipython-input-10-441a4f01e6b3>, line 1)

In [12]:
[*a] = 1,2
a

[1, 2]

In [15]:
(*a,) = 1,2
a

[1, 2]

In [16]:
a = (1,2)
*a

SyntaxError: can't use starred expression here (<ipython-input-16-57adc70a3556>, line 2)

In [17]:
[*a]

[1, 2]

链式比较减少了载入指令，更多基于栈数据复制和交换，会稍稍提升性能，更多的是提升可读性。

In [25]:
dis.dis(compile("1<a<2", '', 'eval'))

  1           0 LOAD_CONST               0 (1)
              2 LOAD_NAME                0 (a)
              4 DUP_TOP
              6 ROT_THREE
              8 COMPARE_OP               0 (<)
             10 JUMP_IF_FALSE_OR_POP    18
             12 LOAD_CONST               1 (2)
             14 COMPARE_OP               0 (<)
             16 RETURN_VALUE
        >>   18 ROT_TWO
             20 POP_TOP
             22 RETURN_VALUE


In [24]:
dis.dis(compile("1<a and a<2", '', 'eval'))

  1           0 LOAD_CONST               0 (1)
              2 LOAD_NAME                0 (a)
              4 COMPARE_OP               0 (<)
              6 JUMP_IF_FALSE_OR_POP    14
              8 LOAD_NAME                0 (a)
             10 LOAD_CONST               1 (2)
             12 COMPARE_OP               0 (<)
        >>   14 RETURN_VALUE


In [30]:
a = list(range(10))
b = a[3:5]
del b
print(a)
del a[3:5]
print(a)
a[2:8] =[1,]
print(a)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 5, 6, 7, 8, 9]
[0, 1, 1]


and返回最后，或导致短路的操作数。or返回第一真值，或最后的操作数。

In [32]:
1 and 2 and 3

3

In [33]:
1 and 2 and 0

0

In [34]:
0 or 1 or 2

1

In [35]:
0 or '' or [] 

[]

In [36]:
"T" if 2 > 1 else "F"

'T'

In [37]:
2 > 1 and "T" or "F"

'T'

In [40]:
2 > 1 and "" or "F" # 有缺陷的情况

'F'

In [39]:
"" if 2 > 1 else "F"

''